In [1]:
import numpy as np
import copy
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
import torch
import torch.nn as nn
import dtnnlib as dtnn
import resnet_cifar

from torchvision import datasets, transforms as T
from torch.utils import data

In [3]:
from tqdm import tqdm
import os, time, sys

In [4]:
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [5]:
cifar_train = T.Compose([
    T.RandomCrop(size=32, padding=4),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize(
        mean=[0.4914, 0.4822, 0.4465], # mean=[0.5071, 0.4865, 0.4409] for cifar100
        std=[0.2023, 0.1994, 0.2010], # std=[0.2009, 0.1984, 0.2023] for cifar100
    ),
])

cifar_test = T.Compose([
    T.ToTensor(),
    T.Normalize(
        mean=[0.4914, 0.4822, 0.4465], # mean=[0.5071, 0.4865, 0.4409] for cifar100
        std=[0.2023, 0.1994, 0.2010], # std=[0.2009, 0.1984, 0.2023] for cifar100
    ),
])

train_dataset = datasets.CIFAR10(root="../../../../../_Datasets/cifar10/", train=True, download=True, transform=cifar_train)
test_dataset = datasets.CIFAR10(root="../../../../../_Datasets/cifar10/", train=False, download=True, transform=cifar_test)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
# train_dataset.data = train_dataset.data.view(-1, 28*28)
# test_dataset.data = test_dataset.data.view(-1, 28*28)

In [7]:
batch_size = 128
train_loader = data.DataLoader(dataset=train_dataset, num_workers=4, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(dataset=test_dataset, num_workers=4, batch_size=batch_size, shuffle=False)

In [8]:
device = torch.device("cuda:1")

In [9]:
criterion = nn.CrossEntropyLoss()

In [10]:
for xx, yy in train_loader:
    xx, yy = xx.to(device), yy.to(device)
    print(xx.shape, yy.shape)
    break

torch.Size([128, 3, 32, 32]) torch.Size([128])


In [11]:
# net = resnet_cifar.cifar_resnet20(num_classes=10, distance=0.5)
# net

In [12]:
# asdasd

## Any function as metric

In [13]:
# class FunctionDT(nn.Module):
    
#     def __init__(self, input_dim, num_centers, func, inv_temp=0.):
#         '''
#         func [input_dim -> 1]
#         '''
#         super().__init__()
#         self.input_dim = input_dim
#         self.num_centers = num_centers
#         self.func = func
        
#         self.inv_temp = nn.Parameter(torch.ones(1)*inv_temp)
        
#         self.centers = torch.randn(num_centers, input_dim)/3.
#         self.centers = nn.Parameter(self.centers)
    
#     def forward(self, x):
#         z = x.unsqueeze(1) - self.centers.unsqueeze(0)
#         dists = self.func(z).squeeze(-1)
#         dists = -dists*torch.exp(self.inv_temp)
#         return dists

In [14]:
# from classes import DistanceRegressor, ConvexNN
# from nflib.flows import SequentialFlow, ActNorm
# import nflib.res_flow as irf

## Try Different metrics for CNN

In [15]:
## Following is copied from 
### https://github.com/kuangliu/pytorch-cifar/blob/master/main.py

# Training
def train(epoch, model, optimizer):
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(tqdm(train_loader)):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    print(f"[Train] {epoch} Loss: {train_loss/(batch_idx+1):.3f} | Acc: {100.*correct/total:.3f} {correct}/{total}")
    return

In [16]:
best_acc = -1
def test(epoch, model, model_name):
    global best_acc
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(tqdm(test_loader)):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
    print(f"[Test] {epoch} Loss: {test_loss/(batch_idx+1):.3f} | Acc: {100.*correct/total:.3f} {correct}/{total}")
    
    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'model': model.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('models'):
            os.mkdir('models')
        torch.save(state, f'./models/{model_name}.pth')
        best_acc = acc

In [17]:
EPOCHS = 200

In [ ]:
acc_dict = {}
for key in ["stereographic", 1]:
    print("_________________________")
    print(f"Experimenting for {key} ;")
    net = resnet_cifar.cifar_resnet20(num_classes=10, distance=key).to(device)
#     net = torch.compile(net)
#     net = torch.compile(net, mode="reduce-overhead")
#     net = torch.compile(net, mode="max-autotune")
    
    model_name = f"00.3_c10_{str(key)}"

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=0.1,
                          momentum=0.9, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
    best_acc = -1
    for epoch in range(EPOCHS):
        train(epoch, net, optimizer)
        test(epoch, net, model_name)
        scheduler.step()
    acc_dict[key] = float(best_acc)
    pass

_________________________
Experimenting for stereographic ;


100%|███████████████████████████████████████████████████| 391/391 [02:03<00:00,  3.17it/s]


[Train] 0 Loss: 1.673 | Acc: 38.060 19030/50000


100%|█████████████████████████████████████████████████████| 79/79 [00:06<00:00, 12.08it/s]


[Test] 0 Loss: 1.733 | Acc: 41.350 4135/10000
Saving..


...

100%|███████████████████████████████████████████████████| 391/391 [02:03<00:00,  3.15it/s]


[Train] 36 Loss: 0.533 | Acc: 81.796 40898/50000


100%|█████████████████████████████████████████████████████| 79/79 [00:06<00:00, 12.06it/s]


[Test] 36 Loss: 1.091 | Acc: 63.640 6364/10000


 42%|█████████████████████▍                             | 164/391 [01:01<01:32,  2.46it/s]

In [ ]:
acc_dict

In [ ]:
# net[0].centers.shape